In [1]:
! pip install python-gdcm
! pip install pylibjpeg pylibjpeg-libjpeg pydicom

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 7.9 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pydicom
import os
import nibabel as nib
import SimpleITK as sitk
from tqdm import tqdm
from tensorflow import keras
from keras import layers, models
import tensorflow_io as tfio
import cv2
import warnings
warnings.filterwarnings('ignore')
from skimage.transform import resize
from tqdm import tqdm

In [3]:
def load_image(path):
    img = pydicom.read_file(path)
    img = img.pixel_array
    return resize(img[:,20:-20], (100, 60))

In [4]:
def window(img):
    img[np.where(img<0)] = 0
    mean = img.mean()
    std = img.std()
    img[np.where(img>(mean+(3*std)))] = (mean+(3*std))
    img = img-img.min()
    maxx = img.max()
    img[np.where(img<(maxx/2))] = maxx/2
    img = (img-(maxx/2)) / (maxx/2)
    return img

In [5]:
e = 0
for p in tqdm(os.listdir("../input/rsna-2022-cervical-spine-fracture-detection/train_images")):
    path = "../input/rsna-2022-cervical-spine-fracture-detection/train_images/"+p+"/"
    len10percent = len(os.listdir(path))//10
    try:
        final_ct = window(load_image(path+str(len10percent)+".dcm")).reshape(100,60,1)
        for image in range(len10percent, len(os.listdir(path))-len10percent):
            img = window(load_image(path+str(image)+".dcm")).reshape(100,60,1)
            final_ct = np.append(final_ct, img, axis=-1)
        resized_final_ct = cv2.resize(final_ct[:,0,:], (100,100)).reshape(100,100,1)
        for i in range(1,final_ct.shape[1]):
            resized_final_ct = np.append(resized_final_ct, cv2.resize(final_ct[:,i,:], (100,100)).reshape(100,100,1), axis=-1)
        ni_img = nib.Nifti1Image(resized_final_ct, affine=np.eye(4))
        nib.save(ni_img, str(p) + ".nii.gz")
    except:
        e=e+1
        print(e)
        print(path)

 18%|█▊        | 371/2019 [44:24<2:40:07,  5.83s/it]

1
../input/rsna-2022-cervical-spine-fracture-detection/train_images/1.2.826.0.1.3680043.17166/


100%|██████████| 2019/2019 [4:12:32<00:00,  7.51s/it]
